In [1]:
import numpy as np
import pandas as pd
import requests
import json
import os

In [4]:
pd.set_option('display.max_colwidth', 200)

# Get Pocket API Key:
def get_API_KEY(pocket_API_key):
    currentDirectory = os.getcwd()
    path_to_API_KEY = currentDirectory + '/' + pocket_API_key
    API_KEY_file = open(path_to_API_KEY)
    
    API_KEY = []
    for i in API_KEY_file:
        API_KEY.append(i)
    
    return str(API_KEY[0])

In [6]:
pocket_API_key = get_API_KEY('pocket_API_key.txt')
print(pocket_API_key)

99707-e5d1302f56f2b5f39fd1ff36


In [7]:
# Get Redirect URL from text file:
def get_REDIRECT_URL(TwitterAccount):
    currentDirectory = os.getcwd()
    RURL_path = currentDirectory + '/' + TwitterAccount
    URL_file = open(RURL_path)
    
    RedirectURL = []
    for i in URL_file:
        RedirectURL.append(i)
    
    return str(RedirectURL[0])

In [8]:
Redirect_URL_key = get_REDIRECT_URL('TwitterAccount.txt')
# print(Redirect_URL_key)

In [9]:
# Authentication Parameters:
auth_params = {'consumer_key': pocket_API_key, 'redirect_uri': Redirect_URL_key}

token = requests.post('https://getpocket.com/v3/oauth/request', data = auth_params)

print('Access code is: ')
print(token.text)

Access code is: 
code=144237a7-f46b-a2e9-3964-eaab33


In [12]:
# Generate the new output link:
auth_Link = 'https://getpocket.com/auth/authorize?request_token=' + token.text.strip('code=') + '&redirect_uri=https%3A//' + Redirect_URL_key.strip('https://')

print('Copy the following link and authorise: ')
print(auth_Link)

Copy the following link and authorise: 
https://getpocket.com/auth/authorize?request_token=144237a7-f46b-a2e9-3964-eaab33&redirect_uri=https%3A//witter.com/vidhya466860832


In [13]:
# Parsing out the access vode from token.text string;
access_code = token.text.split('=')[1]

user_params = {'consumer_key': pocket_API_key, 'code': access_code}

user = requests.post('https://getpocket.com/v3/oauth/authorize', data = user_params)

print('User code is: ')
print(user.text)

User code is: 
access_token=9c64dcd7-5c9b-130a-5778-865fb1&username=sujimithila16%40gmail.com


In [14]:
# Parsing out the access token from the 'user.text' string:
access_token = user.text.split('=')[1].split('&username')[0]

access_token

'9c64dcd7-5c9b-130a-5778-865fb1'

In [15]:
no_articles_params = {'consumer_key': pocket_API_key, 'access_token': access_token, 'tag': 'n'}

no_articles_result = requests.post('https://getpocket.com/v3/get', data = no_articles_params)

no_articles_result.text

'{"status":1,"complete":1,"list":{"1495089268":{"item_id":"1495089268","resolved_id":"1495089268","given_url":"https:\\/\\/medlineplus.gov\\/ency\\/article\\/003211.htm","given_title":"Stress and your health: MedlinePlus Medical Encyclopedia","favorite":"0","status":"0","time_added":"1637422426","time_updated":"1637422433","time_read":"0","time_favorited":"0","sort_id":0,"resolved_title":"Stress and your health","resolved_url":"https:\\/\\/medlineplus.gov\\/ency\\/article\\/003211.htm","excerpt":"Stress is a normal feeling. There are two main types of stress: Your body reacts to stress by releasing hormones. These hormones make your brain more alert, cause your muscles to tense, and increase your pulse.","is_article":"1","is_index":"0","has_video":"0","has_image":"0","word_count":"312","lang":"en","top_image_url":"https:\\/\\/medlineplus.gov\\/images\\/share_ency_1.jpg","listen_duration_estimate":121}},"error":null,"search_meta":{"search_type":"normal"},"since":1638979137}'

In [16]:
no_articles_jf = json.loads(no_articles_result.text)
no_articles_jd = no_articles_jf['list']

In [17]:
# Inspect:
no_articles_jd

{'1495089268': {'item_id': '1495089268',
  'resolved_id': '1495089268',
  'given_url': 'https://medlineplus.gov/ency/article/003211.htm',
  'given_title': 'Stress and your health: MedlinePlus Medical Encyclopedia',
  'favorite': '0',
  'status': '0',
  'time_added': '1637422426',
  'time_updated': '1637422433',
  'time_read': '0',
  'time_favorited': '0',
  'sort_id': 0,
  'resolved_title': 'Stress and your health',
  'resolved_url': 'https://medlineplus.gov/ency/article/003211.htm',
  'excerpt': 'Stress is a normal feeling. There are two main types of stress: Your body reacts to stress by releasing hormones. These hormones make your brain more alert, cause your muscles to tense, and increase your pulse.',
  'is_article': '1',
  'is_index': '0',
  'has_video': '0',
  'has_image': '0',
  'word_count': '312',
  'lang': 'en',
  'top_image_url': 'https://medlineplus.gov/images/share_ency_1.jpg',
  'listen_duration_estimate': 121}}

In [18]:
no_articles_URL = []
for i in no_articles_jd.values():
    url_dat = i.get('resolved_url')
    no_articles_URL.append(url_dat)

no_articles_URL  


['https://medlineplus.gov/ency/article/003211.htm']

In [19]:
no_URL_df = pd.DataFrame(no_articles_URL, columns = ['urls'])

# Create new column called 'wanted':
no_URL_df = no_URL_df.assign(wanted = lambda x: 'n')

no_URL_df

,urls,wanted
0,https://medlineplus.gov/ency/article/003211.htm,n


In [20]:
def get_articles(tag, pocket_API_key, access_token):
    '''
    Func -> Extract article data.
    
    params for this func.:
    tag = 'y' or 'n'
    '''
    articles_params = {'consumer_key': pocket_API_key, 'access_token': access_token, 'tag': tag}
    articles_result = requests.post('https://getpocket.com/v3/get', data = articles_params)
    articles_jf = json.loads(articles_result.text)
    articles_jd = articles_jf['list']
    
    articles_URL = []
    for i in articles_jd.values():
        url_dat = i.get('resolved_url')
        articles_URL.append(url_dat)
        
    URL_df = pd.DataFrame(articles_URL, columns = ['urls'])
    URL_df = URL_df.assign(wanted = lambda x: tag)
    
    return URL_df

In [21]:
yes_URL_df = get_articles('y', pocket_API_key, access_token)

In [22]:
yes_URL_df.head()

,urls,wanted
0,https://www.psychologytoday.com/basics/spirituality,y
1,https://www.indeed.com/career-advice/career-development/improve-your-personal-development-skills,y
2,https://en.wikipedia.org/wiki/Education,y
3,https://www.keepinspiring.me/motivational-quotes/,y


In [23]:
articles_df = pd.concat([yes_URL_df, no_URL_df])
articles_df.dropna(inplace = True)

In [24]:
articles_df

,urls,wanted
0,https://www.psychologytoday.com/basics/spirituality,y
1,https://www.indeed.com/career-advice/career-development/improve-your-personal-development-skills,y
2,https://en.wikipedia.org/wiki/Education,y
3,https://www.keepinspiring.me/motivational-quotes/,y
0,https://medlineplus.gov/ency/article/003211.htm,n


In [25]:
articles_df_copy = articles_df.reset_index()
articles_df_copy

,index,urls,wanted
0,0,https://www.psychologytoday.com/basics/spirituality,y
1,1,https://www.indeed.com/career-advice/career-development/improve-your-personal-development-skills,y
2,2,https://en.wikipedia.org/wiki/Education,y
3,3,https://www.keepinspiring.me/motivational-quotes/,y
4,0,https://medlineplus.gov/ency/article/003211.htm,n


In [26]:
articles_df = articles_df_copy.drop(['index'], axis = 1)
articles_df

,urls,wanted
0,https://www.psychologytoday.com/basics/spirituality,y
1,https://www.indeed.com/career-advice/career-development/improve-your-personal-development-skills,y
2,https://en.wikipedia.org/wiki/Education,y
3,https://www.keepinspiring.me/motivational-quotes/,y
4,https://medlineplus.gov/ency/article/003211.htm,n
